In [2]:
import labhelper as hp
from labhelper.unit_systems.SI import *
import numpy as np
import pandas as pd

In [3]:
Cmass = 431.70 * gram
C = 1.74e3 * joule / kelvin
cwater = 4.184 * joule * kelvin**-1 * gram**-1
a = hp.HelperNew("(m_c * c_p * (T_M - T_C) + C * (T_M - T_C)) / (m_s * (T_B - T_M))", const_in=["c_p"], vars_in=["m_c", "T_C", "T_M", "T_B", "m_s", "C"])
a[["C", "c_p", "m_c_err", "T_C_err", "T_M_err", "T_B_err", "m_s_err", "C_err"]] = [C, cwater, 0.02, 0.1, 0.1, 0.1, 0.02, 0.01]
hot = pd.DataFrame()
hot["m_s"] = [88.36, 90.86, 89.22]
hot["m_s"] = hot["m_s"].mul(gram)
hot["m_c"] = [714.04, 703.58, 681.16]
hot["m_c"] = hot["m_c"].map(lambda x: x*gram-Cmass)
hot["T_B"] = [97, 98, 95.5]
hot["T_C"] = [16.4, 15.6, 14.6]
hot["T_M"] = [21, 17.8, 17.8]
hot[["T_C", "T_B", "T_M"]] = hot[["T_C", "T_B", "T_M"]].map(lambda x: (x+273)*kelvin)
#hot["c_p"] = hot.apply(evaluate, axis=1)
#hot["c_p_err"] = hot.apply(evaluate_err, axis=1)
a.error(hot)

[0.00402799 0.00320668 0.00336331]


[0.00403 J * K^-1 * kg^-1, 0.00321 J * K^-1 * kg^-1, 0.00336 J * K^-1 * kg^-1]

In [9]:
(1 * kelvin)/ kelvin

1 

In [5]:
a = hp.HelperNew("a**2 + b", vars_in=["a", "b"])
a["aerr"] = 0.1
test = pd.DataFrame()
test["a"] = [1, 2]
test["b"] = [1, 1]
test["Δb"] = [0.2, 0.1]
a.error(test)

array([0.08, 0.17])